In [3]:
!apt install python3.10-venv
!python3 -m venv venv

!source venv/bin/activate

!pip install langchain replicate pillow python-decouple

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
0 upgraded, 3 newly installed, 0 to remove and 31 not upgraded.
Need to get 2,473 kB of archives.
After this operation, 2,884 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.4 [1,680 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.1 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.3 [5,716 B]
Fetched 2,473 kB in 0s (6,652 kB/s)
Selecting previously unselected package python3-pip-whl.
(Reading database ... 121730 files and directories currently installed.)
Pr

In [4]:
import os
from decouple import config
from langchain.llms import Replicate

In [5]:
with open(".env", 'w') as file:
    file.write(f"REPLICATE_API_TOKEN=r8_WbvPP4dFYKKMmRXX0Hwod8RVGKgOFji34E7qy")

# load in environment variables
REPLICATE_API_TOKEN = config("REPLICATE_API_TOKEN")
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [6]:
llm: Replicate = Replicate(
    model="meta/codellama-34b-instruct:eeb928567781f4e90d2aba57a51baef235de53f907c214a4ab42adabf5bb9736",
    model_kwargs={"system_prompt": "", "temperature": 0.75, "max_length": 500, "top_p": 1}
)

In [27]:
list_of_topics = "s, Hawaii, Botany, Ferdinand_von_Mueller, Chemist, Politician, John_Macadam, Philosophical_Institute_of_Victoria, Evergreen, Genus, Whorl_(botany), Lanceolate, Obovate, Elliptic, Leaf_shape, Raceme, Tepal, Follicle_(fruit), Seed, Newton_(unit), Hazelnut, Aluminium, Vickers_hardness_test, Glycoside#Cyanogenic_glycosides, Disjunct_distribution, Morphology_(biology), Grafting, Proteaceae, Phytophthora, Macadamia_integrifolia, Macadamia_tetraphylla, Macadamia_tetraphylla, South_Africa, Mediterranean_climate, Temperate_climate, Tropical_climate, Lismore,_New_South_Wales, Macadamia_tetraphylla, Hawaii, William_H._Purvis, Kapulena, Queensland, Gympie, Banana, Carbohydrate, Fat, Protein_(nutrient), Kilocalories, Daily_Value, Essential_nutrients, Thiamine, Vitamin_B6, B_vitamins, Manganese, Iron, Magnesium, Phosphorus, Almond, Cashew, Monounsaturated_fat, Monounsaturated_fat, Omega-7_fatty_acid, Palmitoleic_acid, Food_allergy, Tree_nut_allergy, United_States, Oral_allergy_syndrome, Urticaria, Angioedema, Asthma, Anaphylaxis, Coconut, Walnut, Hazelnut, Cashew, Epinephrine_auto-injector, Toxic, Ingestion, Weakness, Ornamental_plant, Larva, Lepidoptera, Batrachedra, Wikipedia:Citation_needed, Hyacinth_macaw, Parrot"
num_of_topics = 3
goal_topic = "Computer_science"

In [29]:
prompt: str = f"""
From the following list of topics [{list_of_topics}], pick up to {num_of_topics} that relate most closely about {goal_topic}.
Your response should include no explanation. The list of topics MUST be formatted in square brackets [] separated by commas.
DO NOT respond in bullet point lists or numbered lists.
The wording of the topics (including the underscores) must be exactly as given.
"""

response: str = llm(prompt=prompt)
print(response)

  Newton_(unit), Hazelnut, Aluminium


In [8]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import time

In [23]:
class WikiGame:
    def __init__(self, spread_constant=10):
        self.visited_topics = []
        self.path = []
        # Determines how many links it grabs from each wikipedia page
        self.spread_constant = spread_constant

    def start_game(self, llm=False, system_prompt="", temperature=0.75, max_length=500, top_p=1):
        # Asks user for start and end links
        start_link = input("What is the start link?:")
        self.start_topic = re.findall("\/wiki\/([^#]*)", start_link)[0]
        goal_link = input("What is the goal link?:")
        self.goal_topic = re.findall("\/wiki\/([^#]*)", goal_link)[0]

        if llm:
            self.instantiate_llm(system_prompt=system_prompt, temperature=temperature, max_length=max_length, top_p=top_p)

        print("Search Started")
        start_time = time.time()
        # Begins Breadth First Search
        discovered_path = self.bfs_search(llm=llm)

        print("\nPath Found:")
        for topic in discovered_path:
            print(topic)
        print(f"\nTime taken: {time.time()-start_time:.2f} seconds")

    def instantiate_llm(self, system_prompt="", temperature=0.75, max_length=500, top_p=1):
        self.llm: Replicate = Replicate(
            model="meta/codellama-34b-instruct:eeb928567781f4e90d2aba57a51baef235de53f907c214a4ab42adabf5bb9736",
            model_kwargs={"system_prompt": system_prompt, "temperature": temperature, "max_length": max_length, "top_p": top_p}
        )

    def write_token_file(self, token):
        with open(".env", 'w') as file:
            file.write(f"REPLICATE_API_TOKEN={token}")
        # load in environment variables
        REPLICATE_API_TOKEN = config("REPLICATE_API_TOKEN")
        os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

    def bfs_search(self, llm=False):
        # Begins queue
        queue = [(-1, self.start_topic)]

        while queue:
            # Takes first topic in queue
            parent_index, parent_topic = queue.pop(0)

            if parent_topic not in self.visited_topics:
                self.visited_topics.append(parent_topic)
                self.path.append(parent_index)

                # Gets topics from the Wikpedia page
                soup = self.make_soup(parent_topic)
                if llm:
                    topics_found = self.llm_pick_topics(soup)
                else:
                    topics_found = self.find_new_topics(soup)

                print(f"{parent_index} From {self.visited_topics[parent_index]}, searching through: {parent_topic}")

                for topic in topics_found[:self.spread_constant]:
                    queue.append((len(self.visited_topics)-1, topic))
                    if topic == self.goal_topic:
                        # Rebuilds path back to origin topic
                        traversed_topics = [topic, parent_topic]
                        while parent_index >= 0:
                            traversed_topics.append(self.visited_topics[parent_index])
                            parent_index = self.path[parent_index]
                        return traversed_topics[::-1]

    # Creates BeautifulSoup object from wikipedia link
    def make_soup(self, wiki_topic):
        html = urlopen(f"https://en.wikipedia.org/wiki/{wiki_topic}")
        soup = BeautifulSoup(html, "html.parser")
        return soup

    # Finds topics within Wikipedia page
    def find_new_topics(self, soup):
        topics = []
        for link in soup.select('p a[href]'):
            href = link['href']
            if href.startswith('/wiki/'):
                topics.append(href[6:])
        return topics

    def llm_pick_topics(self, soup):
        topic_list = self.find_new_topics(soup)
        print(topic_list)
        string_list = self.list_to_string(topic_list)
        print(string_list)
        prompt = self.get_prompt(self.goal_topic, string_list, self.spread_constant)
        print(prompt)
        response: str = self.llm(prompt=prompt)
        print(response)
        result = self.list_from_string(response, topic_list)
        print(result)
        return result

    def list_to_string(self, topic_list):
        result_string = ""
        for topic in topic_list:
            result_string += str(topic)+", "
        return result_string[:-2]

    def get_prompt(self, goal, string_list, num_of_articles):
        prompt: str = f"""
        From the following list of topics [{string_list}], pick up to {num_of_articles} that relate most closely about {goal}.
        Your response should include no explanation. The list of topics MUST be formatted in square brackets [] separated by commas.
        DO NOT respond in bullet point lists or numbered lists.
        The wording of the topics (including the underscores) must be exactly as given.
        """
        return prompt

    def list_from_string(self, response, topic_list):
        pattern = r"\[(.*)\]"
        match = re.search(pattern, response)
        if match:
            extracted_text = match.group(1)
            extracted_list = [element.strip() for element in extracted_text.split(',')]
            result_list = []
            for element in extracted_list:
                if element in topic_list:
                    result_list.append(element)
            return result_list

In [1]:
game = WikiGame(spread_constant=3)
game.start_game(llm=True)

NameError: name 'WikiGame' is not defined